In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 unidic_lite==1.0.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertJapaneseTokenizer, BertModel
from torch import cuda
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import logging


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
batch_size = 16
max_len = 512

In [5]:
df = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/livedoor_text.csv")
print(df.shape)
df.head()

(7367, 2)


,text,category
0,27日に生放送された日本テレビ「バンクーバー2010」には、女子フィギュアスケートで銀メダル...,7
1,「腐女子」という言葉をご存知でしょうか。\nいわゆる漫画やアニメキャラなどの男性同士の恋愛（...,0
2,展示会イベント恒例のおねいさん写真のコーナーでございます \n\n国内最大級の携帯電話や無線...,6
3,芸能界を引退した島田紳助さんが、今月２８日に公開される映画「犬の首輪とコロッケと」に声だけ出...,2
4,お花に包まれた洋館で、イケメン執事に囲まれながら、ゆったりと過ごす午後のひととき……。女の子...,5


# データセットの作成

In [6]:
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer0, tokenizer1, tokenizer2, tokenizer3, max_len):
    self.X = X
    self.y = y
    self.tokenizers = [tokenizer0, tokenizer1, tokenizer2, tokenizer3]
    self.max_len = max_len

  def __len__(self):
    return len(self.y)

  def encode(self, tokenizer, text):
      inputs = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation = True
      )
      return inputs

  def __getitem__(self, index):
    text = self.X[index]
    label = self.y[index]
    ids = []
    mask = []

    for tokenizer in self.tokenizers:
      inputs = self.encode(tokenizer=tokenizer, text=text)
      ids.append(torch.LongTensor(inputs['input_ids']))
      mask.append(torch.LongTensor(inputs['attention_mask']))

    return {
      'ids': ids,
      'mask': mask,
      'label': label,
      'text':text,
    }

In [7]:
tokenizer0 = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
tokenizer1 = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-char-whole-word-masking")
tokenizer2 = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-char-v2")
tokenizer3 = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")

In [8]:
X = df["text"].values
y = df["category"].values

In [9]:
X_train_eval, X_test, y_train_eval, y_test = train_test_split(X, y, train_size=0.8)

X_train, X_eval, y_train, y_eval = train_test_split(X_train_eval, y_train_eval, train_size=0.75)

print(len(X_train))
print(len(X_eval))
print(len(X_test))

print(len(y_train))
print(len(y_eval))
print(len(y_test))

4419
1474
1474
4419
1474
1474


In [10]:
dataset_train = CreateDataset(X_train, y_train, tokenizer0, tokenizer1, tokenizer2, tokenizer3, max_len=max_len)
dataset_eval = CreateDataset(X_eval, y_eval, tokenizer0, tokenizer1, tokenizer2, tokenizer3, max_len=max_len)
dataset_test = CreateDataset(X_test, y_test, tokenizer0, tokenizer1, tokenizer2, tokenizer3, max_len=max_len)

print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

4419
1474
1474


In [11]:
len(dataset_train[0]["ids"])

4

# データローダの作成

In [12]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, pin_memory=True)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dataloader_train, "val": dataloader_eval}

In [13]:
tmp = next(iter(dataloader_train))
print(tmp["ids"][0].size())
print(tmp["label"])
print(len(tmp["ids"][0]))
print(len(tmp["ids"][1]))
print(len(tmp["ids"][2]))
print(len(tmp["ids"][3]))


torch.Size([16, 512])
tensor([8, 4, 1, 2, 8, 6, 8, 2, 3, 4, 7, 6, 8, 8, 8, 0])
16
16
16
16


# BERTモデル

In [14]:
model0 = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", output_attentions=True, output_hidden_states=True)
model1 = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-char-whole-word-masking", output_attentions=True, output_hidden_states=True)
model2 = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-char-v2", output_attentions=True, output_hidden_states=True)
model3 = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-v2", output_attentions=True, output_hidden_states=True)

In [43]:
from torch import nn

hidden_size = 768

class MultiBertModel(nn.Module):
    '''
    BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル
    '''

    def __init__(self):
        super(MultiBertModel, self).__init__()

        # 日本語学習済みのBERTモデル
        self.berts = [model0, model1, model2, model3] 

        # DropOut
        self.drop = nn.Dropout(0.3)

        # BiLSTM
        self.lstm = torch.nn.LSTM(hidden_size*4, hidden_size, batch_first=True, bidirectional=True)

        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=hidden_size*2, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

        # カウント
        self.count = 0


    def forward(self, input_ids, attention_show_flg:bool):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        attentions = []
        bert_outputs = []
        for i in range(4):
          # BERTの順伝搬
          print(input_ids[i].size())
          print(f"ids_{i} is cuda: {input_ids[i].is_cuda}")
          output = self.berts[i](input_ids[i])
          # 最終4層の隠れ層の状態を取得
          sequence_output = torch.cat([output["hidden_states"][-1*i][:,0] for i in range(1, 5)], dim=1) 

          # BERTの出力を保持
          bert_outputs.append(sequence_output)
          # Attentionの最終層を保持
          attentions.append(output.attentions[-1])
        

        vec = torch.stack((bert_outputs[0], bert_outputs[1], bert_outputs[2], bert_outputs[3]), 1)

        # BiLSTM
        _, lstmout= self.lstm(vec, None)
        bilstm_out = torch.cat([lstmout[0][0], lstmout[0][1]], dim=1)

        # 全結合層
        output = self.cls(self.drop(bilstm_out))  

        self.count += 1

        if attention_show_flg:
          return output, attentions
        else:
          return output


In [44]:
# モデル構築
net = MultiBertModel()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')

ネットワーク設定完了


In [45]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. 各モジュールの一部の層を勾配計算ありに変更
for i in range(1, 5):
  for param in net.berts[0].encoder.layer[-1*i].parameters():
      param.requires_grad = True
  for param in net.berts[1].encoder.layer[-1*i].parameters():
      param.requires_grad = True
  for param in net.berts[2].encoder.layer[-1*i].parameters():
      param.requires_grad = True
  for param in net.berts[3].encoder.layer[-1*i].parameters():
      param.requires_grad = True
  
for param in net.lstm.parameters():
      param.requires_grad =True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True

In [46]:
# 最適化に渡すパラメータ
model_params = [{'params': net.berts[0].encoder.layer[-1*i].parameters(), 'lr': 5e-5} for i in range(1,5)]
model_params.extend([{'params': net.berts[1].encoder.layer[-1*i].parameters(), 'lr': 5e-5} for i in range(1,5)])
model_params.extend([{'params': net.berts[2].encoder.layer[-1*i].parameters(), 'lr': 5e-5} for i in range(1,5)])
model_params.extend([{'params': net.berts[3].encoder.layer[-1*i].parameters(), 'lr': 5e-5} for i in range(1,5)])

lstm_params = [{'params':net.lstm.parameters(),'lr':5e-5}]
model_params.extend(lstm_params)

linear_params = [{'params': net.cls.parameters(), 'lr': 1e-4}]
model_params.extend(linear_params)

In [47]:
# 最適化手法の設定
import torch.optim as optim
optimizer = optim.Adam(model_params)

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 学習・検証

In [48]:
# モデルを学習させる関数を作成
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                ids_for_multi_bert = []
                for i in range(4):
                  ids_for_multi_bert.append(batch["ids"][i].to(device, non_blocking=True))  # 文章
                  # print(len(ids_for_multi_bert))
                  # print(ids_for_multi_bert[0].size())
                
                labels = batch["label"].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = net(ids_for_multi_bert, attention_show_flg=False)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [49]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 1
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
torch.Size([16, 512])
ids_0 is cuda: True


RuntimeError: ignored

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dataloader_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    inputs = batch["ids"][0].to(device)  # 文章
    labels = batch["label"].to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs, attention_show_flg=False)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dataloader_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dataloader_test.dataset), epoch_acc))

100%|██████████| 93/93 [00:25<00:00,  3.63it/s]

テストデータ1474個での正解率：0.9227


# Attentionの可視化

In [ ]:
# BertForIMDbで処理

# ミニバッチの用意
batch = next(iter(dataloader_test))

# GPUが使えるならGPUにデータを送る
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inputs = batch["ids"][0].to(device)  # 文章
labels = batch["label"].to(device)  # ラベル

outputs, attention_probs = net_trained(inputs, attention_show_flg=True)

_, preds = torch.max(outputs, 1)  # ラベルを予測


In [ ]:
attention_probs.size()

torch.Size([16, 12, 512, 512])

In [ ]:
id2label = {
    0: 'dokujo-tsushin', 
    1: 'it-life-hack', 
    2: 'smax', 
    3: 'sports-watch', 
    4: 'kaden-channel', 
    5: 'movie-enter', 
    6: 'topic-news', 
    7: 'livedoor-homme', 
    8: 'peachy'
}

In [ ]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    '''
    Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数
    '''

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights):
    '''
    HTMLデータを作成する
    '''

    # indexの結果を抽出
    sentence = batch["ids"][0][index]  # 文章
    label = batch["label"][index]  # ラベル
    pred = preds[index]  # 予測

    # ラベルと予測結果を文字に置き換え
    label_str = id2label[label.item()]
    pred_str = id2label[pred.item()]

    # 表示用のHTMLを作成する
    html = f"正解ラベル：{label_str}<br>推論ラベル：{pred_str}<br><br>"

    # Self-Attentionの重みを可視化。Multi-Headが12個なので、12種類のアテンションが存在
    for i in range(12):

        # indexのAttentionを抽出と規格化
        # 0単語目[CLS]の、i番目のMulti-Head Attentionを取り出す
        # indexはミニバッチの何個目のデータかをしめす
        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()

        html += '[BERTのAttentionを可視化_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            # 単語が[SEP]の場合は文章が終わりなのでbreak
            if tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
            html += highlight(tokenizer.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"

    # 12種類のAttentionの平均を求める。最大値で規格化
    all_attens = attens*0  # all_attensという変数を作成する
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    html += '[BERTのAttentionを可視化_ALL]<br>'
    for word, attn in zip(sentence, attens):

        # 単語が[SEP]の場合は文章が終わりなのでbreak
        if tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break

        # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
        html += highlight(tokenizer.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
    html += "<br><br>"

    return html


In [ ]:
from IPython.display import HTML

index = 2  # 出力させたいデータ
html_output = mk_html(index, batch, preds, attention_probs)  # HTML作成
HTML(html_output)  # HTML形式で出力
